In [14]:
import gurobipy as g
from gurobipy import *
import bisect
import collections
import numpy as np
import pandas as pd

In [16]:
table_count = int(input('How many tables are there?\n'))
table_size = int(input('How many people can sit at each table?\n'))

How many tables are there?
3
How many people can sit at each table?
2


In [26]:
total_seats = table_count*table_size
#player_count= int(input('How many people are at game night?\n'))
# our start at attempting to implement the bar idea, did not have enough time to implement fully
#if (total_seats<player_count):
#    print("bar created")
#    bar = True
       
table_sizes = [table_size for i in range(table_count)] # creates a list of tables and the number of seats at each
    
#N = sum(table_sizes) also the total_seats value, ie N is the number of seats
#T = len(table_sizes) number of tables
N = total_seats 
T = table_count
table_to_index = collections.defaultdict(list)  # acts as a list with dictionary keys and resulting values, so we can append new values to each key as if each key had a corresponding list

index_to_table = {} #creates a dictionary

played_player = collections.defaultdict(lambda: collections.defaultdict(int)) # acts like a diciontary with keys and then the internal values are also pseudo dictionaries where the values are all ints
acc = list(itertools.accumulate(table_sizes)) 
total_remaining = N*T

#table to index is temp, only used here
#treating each combination of players as an array, with each index as a different seat, index_to_table 
    #is used to move the index of the player in the combination array to the proper table
for x in range(N):
    table = bisect.bisect_right(acc, x)
    table_to_index[table].append(x) 
    index_to_table[x] = table

env = g.Env(empty=True)
env.setParam("OutputFlag",0)
env.start()
m = g.Model(env=env)

idxs = m.addVars(N, name="idxs", vtype=GRB.INTEGER)
played_table = m.addVars(N*T, name="played_table", vtype=GRB.INTEGER)

played_constraints = [None]*(N*T) #played_constraints is a 2D array N*T in size that has a constraint for every player visiting every table
for i in range(N):
    for k in range(T):
        played_constraints[k*N+i] = m.addConstr(played_table[k*N+i] == 0) #initialized every constraint to be = 0

for i in range(N):
    m.addConstr(idxs[i] >= 0) #index of player in combination array must be >= 0
    m.addConstr(idxs[i] <= T-1) #index of player in combination array must be < number of players due to indexing from 0

tbins = [None]*T
for i in range(T):
    tbins[i] = m.addVars(N, name=f"tbins[{i}]", vtype=GRB.BINARY)

for table, size in enumerate(table_sizes):
    for i in range(N):
        m.addConstr((tbins[table][i] == 1) >> (idxs[i] == table))
    m.addConstr(quicksum(tbins[table]) == size)

obj = quicksum(played_table[j*N+i] * tbins[j][i] for i in range(N) for j in range(T))
m.setObjective(obj, GRB.MINIMIZE)
print(*["  Player "  + f"{chr(ord('A') + i)}" for i in range(N)])
mtrx = [] 
while True:
    m.optimize()    
    if (sum(played_table[i*N+j].x == 0 for i in range(T) for j in range(N))):
        for i in range(N):
            mtrx.append(int(idxs[i].x)+1)
        print(*["   Table " + f"{int(idxs[i].x) + 1}" for i in range(N)]) #plus one allows for better reading in context of problem
    else:
        break
    for i in range(N):
        table = int(idxs[i].x)
        m.remove(played_constraints[table*N+i])
        played_constraints[table*N+i] = m.addConstr(played_table[table*N+i] == played_table[table*N+i].x + 1)



tbls = []
for i in range(N): 
    tbls.append(chr(ord('A')+i))
dt = np.array(mtrx)
re_dt = dt.reshape(T, N) # reshapes array as matrix

table_df = pd.DataFrame(re_dt, columns = tbls) 
#table_df['Round'] = [1,2,3] #range(X)
table_df['Round'] = np.arange(1,T+1)


new_df = pd.melt(table_df, id_vars=['Round'], var_name='Person', value_name='Table')
new_df = new_df.sort_values(['Round', 'Table'], ascending=[True, True])

print(new_df.to_string(index=False))

  Player A   Player B   Player C   Player D   Player E   Player F
   Table 1    Table 3    Table 3    Table 2    Table 2    Table 1
   Table 3    Table 1    Table 2    Table 1    Table 3    Table 2
   Table 2    Table 2    Table 1    Table 3    Table 1    Table 3
 Round Person  Table
     1      A      1
     1      F      1
     1      D      2
     1      E      2
     1      B      3
     1      C      3
     2      B      1
     2      D      1
     2      C      2
     2      F      2
     2      A      3
     2      E      3
     3      C      1
     3      E      1
     3      A      2
     3      B      2
     3      D      3
     3      F      3
